# Collect_Twitter_Data

## Install Python libraries

We need the [pymongo](https://pypi.org/project/pymongo/) to manage the MongoDB database, and [tweepy](https://www.tweepy.org/) to call the Twitter APIs.

In [1]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]m1/2 [pymongo]
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tweepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [tweepy]2m2/3 [tweepy]
Note: you may need to restart the kernel to use updated packages.


## Secret Manager Function

In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials  

In [4]:
import pymongo
from pymongo import MongoClient
import json
import tweepy

bearer_token   = get_secret('twitter_api')['bearer_token']

mongodb_connect = get_secret('mongodb')['connection_string']

## Connect to the MongoDB cluster

We will create a database named 'demo' and a collection named 'tweet_collection' in your MongoDB database.

In [5]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

## Use the API to collect tweets

For more about Twitter API 2.0 query operators, please check [Search Tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)

In [7]:
query = 'JMU Football'  #query tweets contain the word of 'election'

Insert the collected Tweets into the MongoDB database. You can set a different max_result, but the max tweets we can collect is 100.

In [8]:
tweet_client = tweepy.Client(bearer_token)

tweets = tweet_client.search_recent_tweets(query=query, max_results=100,
                                    expansions=['author_id'], 
                                    tweet_fields = ['created_at','entities','lang','public_metrics','geo'],
                                    user_fields = ['id', 'location','name', 'public_metrics','username'])

next_token = tweets.meta['next_token']
for user, tweet in zip(tweets.includes['users'],tweets.data):
    tweet_json = {}
    tweet_json['tweet']= tweet.data
    tweet_json['user'] = user.data
    try:
        tweet_collection.insert_one(tweet_json)
        print(tweet_json['tweet']['created_at'])
    except:
        pass

2025-12-05T04:52:25.000Z
2025-12-05T04:39:03.000Z
2025-12-05T04:32:56.000Z
2025-12-05T04:25:16.000Z
2025-12-05T04:23:59.000Z
2025-12-05T03:57:14.000Z
2025-12-05T03:55:48.000Z
2025-12-05T03:54:09.000Z
2025-12-05T03:53:29.000Z
2025-12-05T03:53:19.000Z
2025-12-05T03:52:49.000Z
2025-12-05T03:35:16.000Z
2025-12-05T03:22:05.000Z
2025-12-05T02:58:19.000Z
2025-12-05T02:57:43.000Z
2025-12-05T02:55:42.000Z
2025-12-05T02:39:06.000Z
2025-12-05T02:37:39.000Z
2025-12-05T02:32:22.000Z
2025-12-05T02:26:18.000Z
2025-12-05T02:25:54.000Z
2025-12-05T02:25:30.000Z
2025-12-05T02:14:31.000Z
2025-12-05T02:09:23.000Z
2025-12-05T01:58:29.000Z
2025-12-05T01:57:19.000Z
2025-12-05T01:50:42.000Z
2025-12-05T01:30:07.000Z
2025-12-05T01:26:11.000Z
2025-12-05T01:22:05.000Z
2025-12-05T01:18:59.000Z
2025-12-05T01:07:06.000Z
2025-12-05T01:06:45.000Z
2025-12-05T01:01:45.000Z
2025-12-05T01:00:14.000Z
2025-12-05T00:56:38.000Z
2025-12-05T00:56:31.000Z
2025-12-05T00:49:21.000Z
2025-12-05T00:36:56.000Z
2025-12-05T00:01:27.000Z


Continue fetching early tweets with the same query. YOU WILL REACH YOUR RATE LIMIT VERY FAST!

In [9]:
for i in range(5): # change the number here to fetch more tweets
    tweets = tweet_client.search_recent_tweets(query=query, max_results=100,
                                        expansions=['author_id'], 
                                        tweet_fields = ['created_at','entities','lang','public_metrics','geo'],
                                        user_fields = ['id', 'location','name', 'public_metrics','username'],
                                        next_token=next_token)
    next_token = tweets.meta['next_token']
    for user, tweet in zip(tweets.includes['users'],tweets.data):
        tweet_json = {}
        tweet_json['tweet']= tweet.data
        tweet_json['user'] = user.data
        try:
            tweet_collection.insert_one(tweet_json)
            print(tweet_json['tweet']['created_at'])
        except:
            pass

TooManyRequests: 429 Too Many Requests
Too Many Requests

## Close Database Connection

In [10]:
mongo_client.close()